In [1]:
import openturns as ot
import openturns.viewer as otv
import salib
import treemap as tm
import pickle
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi']= 500
plt.rcParams['text.usetex'] = True

In [2]:
with open("../../data/PCE_metamodel.pkl", "rb") as f:
    polynomialChaosResult = pickle.load(f)

In [3]:
# %% Gather intermediate data
inputSample = polynomialChaosResult.getInputSample()
inputDimension = inputSample.getDimension()

In [4]:
# Get input description
inputDistribution = polynomialChaosResult.getDistribution()
inputDescription = inputDistribution.getDescription()

In [5]:
# %%
chaosSI = ot.FunctionalChaosSobolIndices(polynomialChaosResult)

In [6]:
# %%
group_list, interaction_sobol_group_list = salib.ComputeAllSobolInteractionIndices(polynomialChaosResult)
print("Before threshold:")
print(f"Number of groups = {len(group_list)}, "
      f"number of interaction indices = {len(interaction_sobol_group_list)}")

Before threshold:
Number of groups = 18, number of interaction indices = 18


In [7]:
def CreateMosaicPlot(polynomialChaosResult, marginalIndex = 0, maximumNumberOfColumns = 4, 
                     sensitivity_threshold = 0.005, verbose = False):
    group_list, interaction_sobol_group_list = salib.ComputeAllSobolInteractionIndices(polynomialChaosResult, marginalIndex)
    if verbose:
        print("Before threshold:")
        print(f"Number of groups = {len(group_list)}, "
            f"number of interaction indices = {len(interaction_sobol_group_list)}")

    # Set a threshold on the interaction indice
    if verbose:
        print(f"Number of groups before threshold = {len(group_list)}")
    
    if verbose:
        print("Threshold = ", sensitivity_threshold)
    interaction_sensitivity_threshold_list = []
    groups_threshold_list = []
    for i in range(len(group_list)):
        group = group_list[i]
        value = interaction_sobol_group_list[i]
        if verbose:
            print("group = ", group, " : ", value)
        if value > sensitivity_threshold:
            groups_threshold_list.append(group)
            interaction_sensitivity_threshold_list.append(value)

    if verbose:
        print("After threshold:")
        print(f"Number of groups = {len(groups_threshold_list)}, "
            f"number of interaction indices = {len(interaction_sensitivity_threshold_list)}")

    # Compute labels
    group_labels_with_description = salib.ComputeGroupLabelsFromLabelNames(
        inputDescription, groups_threshold_list
    )
    if verbose:
        print(f"Group labels (nb. = {len(group_labels_with_description)}) = {group_labels_with_description}")

    # Add a Joker index representing ignored values
    interaction_sensitivity_remainder = 1.0 - sum(interaction_sensitivity_threshold_list)
    if verbose:
        print(f"Remainder = {interaction_sensitivity_remainder:.4f}")
    interaction_sensitivity_threshold_list.append(interaction_sensitivity_remainder)
    group_labels_with_description.append("*")

    if verbose:
        print("After threshold:")
        print(f"Number of groups = {len(group_labels_with_description)}, "
            f"number of interaction indices = {len(interaction_sensitivity_threshold_list)}")


    # Plot treemap
    treemap = tm.TreeMap(
        interaction_sensitivity_threshold_list,
        group_labels_with_description,
        sum_threshold=interaction_sensitivity_remainder,
        text_size=0.85
    )
    
    best_items_by_column = treemap.searchForBestNumberOfItemsByLine(maximumNumberOfColumns)
    graph = treemap.columnwise(best_items_by_column)
    graph.setTitle(
        "Output index = %d, dim. = %d, remainder = %.4f" % (marginalIndex, inputDimension, interaction_sensitivity_remainder)
    )
    return graph


In [16]:
# %%
plotSelection = True
if plotSelection:
    marginalOutputIndexList = [15, 33, 59]
else:
    outputSample = polynomialChaosResult.getOutputSample()
    outputDimension = outputSample.getDimension()
    marginalOutputIndexList = list(range(outputDimension))

for marginalIndex in marginalOutputIndexList:
    graph = CreateMosaicPlot(polynomialChaosResult, marginalIndex)
    a = graph.getDrawables()
    for el in a:
    #view = otv.View(graph, square_axes=True, figure_kw={"figsize": (7.0, 7.0)})
    #plt.subplots_adjust(top = 0.95)
    #figure = view.getFigure()
    #base_filename = "plot_sensitivity_mosaique_thycgv_%d" % (marginalIndex)
    #figure.savefig(base_filename + ".pdf", bbox_inches="tight")
    #figure.savefig(base_filename + ".png", bbox_inches="tight", dpi=90)

# %%

Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.216793 0.5      ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.565429 0.5      ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.79834 0.5     ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.949704 0.253884 ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.949704 0.602373 ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.949704 0.766562 ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.949704 0.884754 ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.949704 0.966681 ]
Text(name=Unnamed, color=blue, fill=solid, line=solid, point=plus, width=1, data=
0 : [ 0.202534 0.5      ]
Text(name=Unnamed, color=blue,

In [18]:
len(a)

16